In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

# Wykrycie kuli rzułtych (ang. balls)

In [32]:
def process_frame(current_frame: np.ndarray, lower_range:np.ndarray = None, upper_range:np.ndarray = None) -> np.ndarray:
    """Process single frame and return new frame with detected objects."""
    original_image: np.ndarray = current_frame.copy()
    
    hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)
    if lower_range is None:
        lower_range = np.array([0, 90, 160])
    if upper_range is None:
        upper_range = np.array([30, 255, 255])
    binary_image = cv2.inRange(hsv_image, lower_range, upper_range)
    opened_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Add a black rectangle to the upper right quarter of the image opened_image
    height, width, _ = current_frame.shape
    opened_image[0:height//2, width//2:width] = 0

    # Find contours and add rectangles to the original image

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened_image)
    
    for i in range(1, num_labels):
        areas = stats[:, -1]  # Get the areas of all labels
        top_labels = np.argsort(areas)[::-1][1:6]  # Get the indices of the top 5 labels with the largest areas

        for i in top_labels:
            x, y, w, h, area = stats[i]
            if area > 3:
                cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(original_image, f'{area}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return original_image


def process_video(file: str, lower = None, upper = None) -> None:
    """Process video file and display it with detected objects."""
    capture = cv2.VideoCapture(file)

    while capture.isOpened():
        has_next_frame, current_frame = capture.read()
        if not has_next_frame:
            break

        cv2.waitKey(5)
        new_image = process_frame(current_frame, lower, upper)
        cv2.imshow('Processed video', new_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()



In [33]:
yellow_lower = np.array([0, 90, 160])
yellow_upper = np.array([30, 255, 255])

red_lower = np.array([150, 80, 160])
red_upper = np.array([179, 255, 255])

In [34]:
process_video('mats/video_2.mp4', red_lower, red_upper)